Hi gens! My idea is applying **Coulomb Interaction** which force is propotional to inversed squared distance (1/r^2). Inversed distance (1/r) can be applicable when focusing on potential. Anyway, i've considered inversed squared distance, here.

1. get assigned atoms from train data which are included 'atom_index_0' or 'atom_index_1' of molecule
2. get distances from each atom belonging to the molecule and pickup 'num = 5' nearest regarding to each atom [H, C, N, O, F]. Though in this competition we focus on bondings of H-H, H-C, H-N, properties of bondings are strongly affected O, F. That's why I'd like to consider interaction as I mentioned.
3. mearge distance array according to atom_index_0 and atom_index_1 then dimension of feature of bonding is 50 = num x atoms x 2.
4. feed the feature into model. model can be built for each bonding type, 1JHH, 1JHC, 2JHC etc.

In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import random
import lightgbm as lgb

from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPRegressor

import pickle

In [2]:
FOLDER = '../../data_kaggle/champs/'
OUTPUT = FOLDER + 'out/'
os.listdir(FOLDER)

['mulliken_charges.csv',
 'sample_submission.csv',
 'magnetic_shielding_tensors.csv',
 'train.csv',
 'test.csv',
 'dipole_moments.csv',
 'potential_energy.csv',
 'structures.csv',
 'out',
 'scalar_coupling_contributions.csv',
 'dsgdb9nsd_000001.xyz',
 'structures.zip']

In [3]:
# df_mulliken_charges = pd.read_csv(FOLDER + 'mulliken_charges.csv')
# df_sample =  pd.read_csv(FOLDER + 'sample_submission.csv')
# df_magnetic_shielding_tensors = pd.read_csv(FOLDER + 'magnetic_shielding_tensors.csv')
# df_train = pd.read_csv(FOLDER + 'train.csv')
df_test = pd.read_csv(FOLDER + 'test.csv')
# df_dipole_moments = pd.read_csv(FOLDER + 'dipole_moments.csv')
# df_potential_energy = pd.read_csv(FOLDER + 'potential_energy.csv')
df_structures = pd.read_csv(FOLDER + 'structures.csv')
# df_scalar_coupling_contributions = pd.read_csv(FOLDER + 'scalar_coupling_contributions.csv')

to find target bonding

In [4]:
def assign_atoms_index(df, molecule):
    se_0 = df.query('molecule_name == "{}"'.format(molecule))['atom_index_0']
    se_1 = df.query('molecule_name == "{}"'.format(molecule))['atom_index_1']
    assign_idx = pd.concat([se_0, se_1]).unique()
    assign_idx.sort()
    return assign_idx

In [5]:
def get_dist_matrix(df_structures, molecule):
    df_temp = df_structures.query('molecule_name == "{}"'.format(molecule))
    locs = df_temp[['x','y','z']].values
    num_atoms = len(locs)
    loc_tile = np.tile(locs.T, (num_atoms,1,1))
    dist_mat = ((loc_tile - loc_tile.T)**2).sum(axis=1)
    return dist_mat

In [ ]:
def get_pickup_dist_matrix(df, df_structures, molecule, num_pickup=5, atoms=['H', 'C', 'N', 'O', 'F']):
    pickup_dist_matrix = np.zeros([0, len(atoms)*num_pickup*2])
    assigned_idxs = assign_atoms_index(df, molecule) # [0, 1, 2, 3, 4, 5, 6] -> [1, 2, 3, 4, 5, 6]
    dist_mat = get_dist_matrix(df_structures, molecule)
    for idx in assigned_idxs: # [1, 2, 3, 4, 5, 6] -> [2]
        df_temp = df_structures.query('molecule_name == "{}"'.format(molecule))
        locs = df_temp[['x','y','z']].values

        dist_arr = dist_mat[idx] # (7, 7) -> (7, )

        atoms_mole = df_structures.query('molecule_name == "{}"'.format(molecule))['atom'].values # ['O', 'C', 'C', 'N', 'H', 'H', 'H']
        atoms_mole_idx = df_structures.query('molecule_name == "{}"'.format(molecule))['atom_index'].values # [0, 1, 2, 3, 4, 5, 6]

        mask_atoms_mole_idx = atoms_mole_idx != idx # [ True,  True, False,  True,  True,  True,  True]
        masked_atoms = atoms_mole[mask_atoms_mole_idx] # ['O', 'C', 'N', 'H', 'H', 'H']
        masked_atoms_idx = atoms_mole_idx[mask_atoms_mole_idx]  # [0, 1, 3, 4, 5, 6]
        masked_dist_arr = dist_arr[mask_atoms_mole_idx]  # [ 5.48387003, 2.15181049, 1.33269675, 10.0578779, 4.34733927, 4.34727838]
        masked_locs = locs[masked_atoms_idx]

        sorting_idx = np.argsort(masked_dist_arr) # [2, 1, 5, 4, 0, 3]
        sorted_atoms_idx = masked_atoms_idx[sorting_idx] # [3, 1, 6, 5, 0, 4]
        sorted_atoms = masked_atoms[sorting_idx] # ['N', 'C', 'H', 'H', 'O', 'H']
        sorted_dist_arr = 1/masked_dist_arr[sorting_idx] #[0.75035825,0.46472494,0.23002898,0.23002576,0.18235297,0.09942455]
        sorted_locs = masked_locs[sorting_idx]
        
        nearest_idx = sorted_atoms_idx[0]
        nearest_atom = sorted_atoms[0]
        
        base_vec = sorted_locs[0] - locs[idx]
        base_vec = base_vec / np.linalg.norm(base_vec)

        target_matrix = np.zeros([len(atoms), num_pickup*2])
        for a, atom in enumerate(atoms):
            pickup_atom = sorted_atoms == atom # [False, False,  True,  True, False,  True]
            pickup_dist = sorted_dist_arr[pickup_atom] # [0.23002898, 0.23002576, 0.09942455]
            pickup_locs = sorted_locs[pickup_atom] - np.tile(locs[idx], (sum(pickup_atom), 1))
            
            in_prods = np.dot(base_vec, pickup_locs.T) / np.linalg.norm(pickup_locs, axis=1)

            num_atom = len(pickup_dist)
            if num_atom > num_pickup:
                target_matrix[a, :num_pickup] = pickup_dist[:num_pickup]
                target_matrix[a, num_pickup:] = in_prods[:num_pickup]
            else:
                target_matrix[a, :num_atom] = pickup_dist
                target_matrix[a, num_pickup:num_pickup+num_atom] = in_prods
        
        pickup_dist_matrix = np.vstack([pickup_dist_matrix, target_matrix.reshape(-1)])
    return pickup_dist_matrix

In [ ]:
num = 5
mols = df_test['molecule_name'].unique()
num_div = len(mols) // 5
dist_mat = np.zeros([0, num*5*2])
atoms_idx = np.zeros([0], dtype=np.int32)
molecule_names = np.empty([0])

start = time.time()

for mol in mols[:num_div]:
    
    assigned_idxs = assign_atoms_index(df_test, mol)
    dist_mat_mole = get_pickup_dist_matrix(df_test, df_structures, mol, num_pickup=num)
    mol_name_arr = [mol] * len(assigned_idxs) 
    
    molecule_names = np.hstack([molecule_names, mol_name_arr])
    atoms_idx = np.hstack([atoms_idx, assigned_idxs])
    dist_mat = np.vstack([dist_mat, dist_mat_mole])
    
col_name_list = []
atoms = ['H', 'C', 'N', 'O', 'F']
for a in atoms:
    for n in range(num):
        col_name_list.append('dist_{}_{}'.format(a, n))
    for n in range(num):
        col_name_list.append('cos_{}_{}'.format(a, n))
        
se_mole = pd.Series(molecule_names, name='molecule_name')
se_atom_idx = pd.Series(atoms_idx, name='atom_index')
df_dist = pd.DataFrame(dist_mat, columns=col_name_list)
df_distance = pd.concat([se_mole, se_atom_idx,df_dist], axis=1)

elapsed_time = time.time() - start
print ("elapsed_time:{0}".format(elapsed_time) + "[sec]")

In [ ]:
df_distance.to_csv(OUTPUT + '20190611_distance_test_cos0.csv', index=False)

In [9]:
df_distance.head()

,molecule_name,atom_index,dist_H_0,dist_H_1,dist_H_2,dist_H_3,dist_H_4,cos_H_0,cos_H_1,cos_H_2,...,dist_F_0,dist_F_1,dist_F_2,dist_F_3,dist_F_4,cos_F_0,cos_F_1,cos_F_2,cos_F_3,cos_F_4
0,dsgdb9nsd_000004,0,0.886482,0.195583,0.000000,0.000000,0.000000,1.0,-1.0000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,dsgdb9nsd_000004,1,0.886482,0.195583,0.000000,0.000000,0.000000,1.0,-1.0000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,dsgdb9nsd_000004,2,0.090546,0.000000,0.000000,0.000000,0.000000,1.0,0.0000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,dsgdb9nsd_000004,3,0.090546,0.000000,0.000000,0.000000,0.000000,1.0,0.0000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,dsgdb9nsd_000015,0,0.837293,0.822961,0.822959,0.143484,0.143451,1.0,-0.3224,-0.322423,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
df_distance1 = pd.read_csv(OUTPUT + '20190611_distance_test_cos1.csv')
df_distance2 = pd.read_csv(OUTPUT + '20190611_distance_test_cos2.csv')

In [11]:
df_dist = pd.concat([df_distance, df_distance1, df_distance2], axis=0)
df_dist.shape

(756113, 52)

In [ ]:
df_train_dist = pd.read_csv(OUTPUT + '20190611_train_dist_cos.csv')

In [ ]:
bond_types = df_train_dist['type'].unique()

In [ ]:
del df_train_dist

In [12]:
def merge_atom(df, df_distance):
    df_merge_0 = pd.merge(df, df_distance, left_on=['molecule_name', 'atom_index_0'], right_on=['molecule_name', 'atom_index'])
    df_merge_0_1 = pd.merge(df_merge_0, df_distance, left_on=['molecule_name', 'atom_index_1'], right_on=['molecule_name', 'atom_index'])
    del df_merge_0_1['atom_index_x'], df_merge_0_1['atom_index_y']
    return df_merge_0_1

In [13]:
start = time.time()
df_test_dist = merge_atom(df_test, df_dist)
elapsed_time = time.time() - start
print ("elapsed_time:{0}".format(elapsed_time) + "[sec]")

elapsed_time:2.6169302463531494[sec]


In [14]:
df_test_dist.to_csv(OUTPUT + '20190611_test_dist_cos.csv', index=False)

In [16]:
df_test_dist.head(10)

,id,molecule_name,atom_index_0,atom_index_1,type,dist_H_0_x,dist_H_1_x,dist_H_2_x,dist_H_3_x,dist_H_4_x,...,dist_F_0_y,dist_F_1_y,dist_F_2_y,dist_F_3_y,dist_F_4_y,cos_F_0_y,cos_F_1_y,cos_F_2_y,cos_F_3_y,cos_F_4_y
0,4658147,dsgdb9nsd_000004,2,0,2JHC,0.090546,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,4658150,dsgdb9nsd_000004,3,0,1JHC,0.090546,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,4658148,dsgdb9nsd_000004,2,1,1JHC,0.090546,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4658151,dsgdb9nsd_000004,3,1,2JHC,0.090546,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4658149,dsgdb9nsd_000004,2,3,3JHH,0.090546,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,4658152,dsgdb9nsd_000015,3,0,1JHC,0.314568,0.313845,0.164981,0.108185,0.076110,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,4658156,dsgdb9nsd_000015,4,0,1JHC,0.314568,0.313851,0.164998,0.108267,0.076114,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,4658159,dsgdb9nsd_000015,5,0,1JHC,0.313851,0.313845,0.076113,0.076111,0.061192,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,4658161,dsgdb9nsd_000015,6,0,3JHC,0.314568,0.313845,0.164998,0.108185,0.076113,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,4658165,dsgdb9nsd_000015,7,0,3JHC,0.314568,0.313851,0.164981,0.108267,0.076111,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [18]:
df_structures.query('molecule_name == "dsgdb9nsd_000004"')

,molecule_name,atom_index,atom,x,y,z
12,dsgdb9nsd_000004,0,C,0.599539,0.0,1.0
13,dsgdb9nsd_000004,1,C,-0.599539,0.0,1.0
14,dsgdb9nsd_000004,2,H,-1.661639,0.0,1.0
15,dsgdb9nsd_000004,3,H,1.661639,0.0,1.0
